In [2]:
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
import time
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv('E:\SnehaWork\AIProject\Dataset\Customer-Churn.csv')
df.columns[df.isnull().any()].tolist() 

[]

In [4]:
Churned = df[df.Churn == 'Yes']
NonChurned = df[df.Churn == 'No']

number_of_Churned = len(Churned)
number_of_NonChurned = len(NonChurned)

Churned_percentage = round((number_of_Churned / (number_of_Churned + number_of_NonChurned)) * 100, 2)
valid_percentage = round((number_of_NonChurned / (number_of_Churned + number_of_NonChurned)) * 100, 2)
print(number_of_NonChurned,number_of_Churned)
#df.columns.tolist()

5174 1869


In [5]:
#df.dtypes
#count yes and no on base data, train and test

In [7]:
df.drop('customerID',axis='columns',inplace=True)

In [9]:
#error='coerce' ignores the errors. Conversion for all except for error values i.e. empty spaces. 
#.isnull() gives columns which have empty spaces
#shows the data which has TotalCharges as null. Good to drop these
df.columns[df.isnull().any()].tolist() 
df[pd.to_numeric(df.TotalCharges,errors='coerce').isnull()].shape

(11, 20)

In [10]:
#Drop rows with TotalCharges as null

df1 = df[df.TotalCharges!=' ']

In [11]:
df1.TotalCharges = pd.to_numeric(df1.TotalCharges)

C:\Users\USER\Anaconda3\lib\site-packages\pandas\core\generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [12]:
df1.replace('No internet service','No',inplace=True)
df1.replace('No phone service','No',inplace=True)

C:\Users\USER\Anaconda3\lib\site-packages\pandas\core\frame.py:4263: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  method=method,


In [13]:
#replace Yes/No to 1/0 since ML doesn't understand text

yes_no_columns = ['Partner','Dependents','PhoneService','MultipleLines','OnlineSecurity','OnlineBackup',
                  'DeviceProtection','TechSupport','StreamingTV','StreamingMovies','PaperlessBilling','Churn']

for col in yes_no_columns:
    df1[col].replace({'Yes': 1,'No': 0},inplace=True)

C:\Users\USER\Anaconda3\lib\site-packages\pandas\core\generic.py:6786: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [14]:
df1['gender'].replace({'Male':1,'Female':0},inplace=True)

In [15]:
#One hot encoding for multiclass i.e. InternetService, Contract, PaymentMethod

multiclass_col=['InternetService', 'Contract', 'PaymentMethod']
df1 = pd.get_dummies(data=df1,columns=multiclass_col)

In [16]:
#scaling the values between 1 and 0 for below
cols_to_scale = ['tenure','MonthlyCharges','TotalCharges']

#MinMaxScaler scales the data between 0 and 1
scaler = MinMaxScaler()

df1[cols_to_scale] = scaler.fit_transform(df1[cols_to_scale])

In [17]:
def print_unique_col_values(df):
    for column in df:
        print(f'{column} : {df[column].unique()}')

In [18]:
print_unique_col_values(df1)

gender : [0 1]
SeniorCitizen : [0 1]
Partner : [1 0]
Dependents : [0 1]
tenure : [0.         0.46478873 0.01408451 0.61971831 0.09859155 0.29577465
 0.12676056 0.38028169 0.85915493 0.16901408 0.21126761 0.8028169
 0.67605634 0.33802817 0.95774648 0.71830986 0.98591549 0.28169014
 0.15492958 0.4084507  0.64788732 1.         0.22535211 0.36619718
 0.05633803 0.63380282 0.14084507 0.97183099 0.87323944 0.5915493
 0.1971831  0.83098592 0.23943662 0.91549296 0.11267606 0.02816901
 0.42253521 0.69014085 0.88732394 0.77464789 0.08450704 0.57746479
 0.47887324 0.66197183 0.3943662  0.90140845 0.52112676 0.94366197
 0.43661972 0.76056338 0.50704225 0.49295775 0.56338028 0.07042254
 0.04225352 0.45070423 0.92957746 0.30985915 0.78873239 0.84507042
 0.18309859 0.26760563 0.73239437 0.54929577 0.81690141 0.32394366
 0.6056338  0.25352113 0.74647887 0.70422535 0.35211268 0.53521127]
PhoneService : [0 1]
MultipleLines : [0 1]
OnlineSecurity : [0 1]
OnlineBackup : [1 0]
DeviceProtection : [0 1]
Tech

In [19]:
#Drop churn 
x = df1.drop('Churn',axis=1)  #check axis=0/1
y = df1['Churn']
y.value_counts()

0    5163
1    1869
Name: Churn, dtype: int64

In [20]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=15)

In [21]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(sampling_strategy='minority')
X_sm_train, y_sm_train = sm.fit_resample(X_train, y_train)


Using TensorFlow backend.


In [52]:
X_test = pd.DataFrame(X_test, columns=x.columns)
#X_train = pd.DataFrame(X_train_oversampled, columns=X_train.columns)


ValueError: Must pass 2-d input

In [23]:
y_sm_train = pd.Series(y_sm_train)


In [24]:
#X_sm_train, X_sm_val, y_sm_train, y_sm_val = train_test_split(X_sm_train,y_sm_train,test_size=0.2,random_state=5)

In [25]:
num_of_cols = len(x.columns)
num_of_cols

26

In [40]:
#X_sm_train=X_sm_train.values.reshape(X_sm_train.shape[0],num_of_cols, 1).astype('float32') 
X_test=X_test.reshape(X_test.shape[0],num_of_cols, 1).astype('float32')
#X_sm_val=X_sm_val.values.reshape(X_sm_val.shape[0],num_of_cols, 1).astype('float32')

#sometimes works as X_val.values.reshape(X_val.shape[0],26, 1).astype('float32')

In [27]:
X_sm_train.shape


(8246, 26, 1)

In [28]:
#X_sm_val.shape


In [29]:
X_test.shape

(1407, 26)

In [30]:
Name = "CNN_{}".format(int(time.time())) 
log_dir="E:\\SnehaWork\\AIProject\\TF_Logs\\{}".format(Name)
cnn = models.Sequential([
    layers.Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=(26,1)),  
    layers.MaxPooling1D(pool_size=3),
    
    layers.Conv1D(filters=128, kernel_size=3, activation='relu'),
    layers.MaxPooling1D(pool_size=3),
    
    layers.Flatten(),
    layers.Dense(20, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

#cnn.summary()
tb_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir,histogram_freq=0)

cnn.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

cnn.fit(X_sm_train, y_sm_train, epochs=10, callbacks=[tb_callback],verbose=1,validation_split=0.2)

Train on 6596 samples, validate on 1650 samples
Epoch 1/10
6596/6596 [==============================] - 5s 760us/sample - loss: 0.5135 - accuracy: 0.7445 - val_loss: 0.8552 - val_accuracy: 0.5127
Epoch 2/10
6596/6596 [==============================] - 3s 435us/sample - loss: 0.4732 - accuracy: 0.7699 - val_loss: 0.6834 - val_accuracy: 0.6970
Epoch 3/10
6596/6596 [==============================] - 3s 426us/sample - loss: 0.4635 - accuracy: 0.7737 - val_loss: 0.6857 - val_accuracy: 0.6582
Epoch 4/10
6596/6596 [==============================] - 3s 433us/sample - loss: 0.4595 - accuracy: 0.7743 - val_loss: 0.6111 - val_accuracy: 0.7079
Epoch 5/10
6596/6596 [==============================] - 3s 436us/sample - loss: 0.4520 - accuracy: 0.7812 - val_loss: 0.5737 - val_accuracy: 0.7273
Epoch 6/10
6596/6596 [==============================] - 3s 425us/sample - loss: 0.4484 - accuracy: 0.7811 - val_loss: 0.7197 - val_accuracy: 0.6618
Epoch 7/10
6596/6596 [==============================] - 3s 435us

In [41]:
cnn.evaluate(X_test,y_test)

1407/1407 [==============================] - 0s 132us/sample - loss: 0.5632 - accuracy: 0.7001


[0.5631817793625843, 0.7000711]

In [55]:
yp_pred= cnn.predict(X_test)


In [56]:
yp_pred = np.round(yp_pred)

In [57]:
from sklearn.metrics import confusion_matrix , classification_report

print("Classification Report: \n", classification_report(y_test, yp_pred))


Classification Report: 
               precision    recall  f1-score   support

           0       0.87      0.70      0.77      1040
           1       0.45      0.71      0.55       367

    accuracy                           0.70      1407
   macro avg       0.66      0.70      0.66      1407
weighted avg       0.76      0.70      0.72      1407



In [58]:
tf.math.confusion_matrix(labels=y_test,predictions=yp_pred)


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[723, 317],
       [105, 262]])>